# Subsetting GEDI L4A Example

Optional packages to be installed: `geopandas`, `contextily`, `backoff`

`geopandas` and `contextily` are used to plot both the aoi and results from 
gedi_l4a_subsetting to verify that both are as expected. `backoff` is used to 
recursively call a function until a true response is received which is useful when
getting job results.

Job submissions are still operable without the optional packages. Skip *optional*
cells and verify the results of the job using your tool of choice.


## 1. Define Area of Interest

If your AOI is a publicly available geoBoundary, see
[Getting the GeoJSON URL for a geoBoundary](#getting-the-geojson-url-for-a-geoboundary)
for details on obtaining it's URL.  In this case, that is the URL you must
supply for the `aoi` input.

Alternatively, you can make your own GeoJSON file for your AOI and place it
within your public bucket within the ADE.  Based upon where you place your
GeoJSON file, you can construct a URL to specify for the a job's `aoi` input
where `path/to/aio.geojson` can be any path and filename for your
AOI within `my-public-bucket`.

You would then supply the following URL as the `aoi` input value when running
this algorithm as a DPS job, where `<USERNAME>` is your ADE username and where
`path/to/aio.geojson` is the location of your GeoJSON file within `my-public-bucket`:


```
aoi = "https://maap-ops-workspace.s3.amazonaws.com/shared/<USERNAME>/path/to/aoi.geojson"
```


In [ ]:
from maap.maap import MAAP

maap = MAAP(maap_host="api.ops.maap-project.org")
username = maap.profile.account_info()["username"]
aoi = f"{username}/path/to/aoi.geojson"

In [ ]:
# (Optional Cell) Verify the contents of the area of interest
import geopandas as gpd
import contextily as ctx

aoi = gpd.read_file(f'/projects/shared-buckets/{aoi}')
aoi_epsg4326 = aoi.to_crs(epsg=4326) # Convert if not already
ax=aoi_epsg4326.plot(figsize=(10, 5), alpha=0.3, edgecolor='red')
ctx.add_basemap(ax, crs=4326)

## 2. Submit a Job

When supplying input values, to use the default value (for the select fields below), enter a dash (`-`) as the input value.


- `aoi` (required): URL[str]: URL to a GeoJSON file representing your area of interest.
See [above](#1.-Define-Area-of-Interest) for specifics.

- `columns`: Sequence[str]: Comma-separated list of column names to include in output file.
    `(Default: agbd, agbd_se, l2_quality_flag, l4_quality_flag, sensitivity, sensitivity_a2)`

- `query`: str: Query expression for subsetting the rows in the output file.
<b>IMPORTANT</b>: The columns input must contain at least all of the columns that appear in this query expression, otherwise an error will occur.
`(Default: "l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95 and sensitivity_a2 > 0.95")`

- `limit`: Maximum number of GEDI granule data files to download (among those that intersect the specified AOI). `(Default: 10,000)`

In [ ]:
# Submitting a DPS job
# Sets 'result_job_id' to use with maap.getJobResult
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset_ubuntu",
    version="gedi-subset-0.2.4",
    queue="maap-dps-worker-8gb",
    username=username,
    aoi=f"https://maap-ops-workspace.s3.amazonaws.com/shared/{aoi}",
    columns="-",
    query="-",
    limit="-",
)
result_job_id = result["job_id"]
print(result)

In [ ]:
# (Optional) Recursively call maap.getJobResult until a response 200 is received
# Sets 'location' to path in my-private-bucket in ADE
import backoff
import re
import xml.etree.ElementTree as ET

@backoff.on_predicate(backoff.expo, max_value=60)
def get_result(result_job_id):
    result = maap.getJobResult(result_job_id)
    if (result.status_code == 200):
        root = ET.fromstring(result.text)
        locations = [element.text for element in root.findall('.//{http://www.opengis.net/wps/2.0}Data')]
        print(locations)

        search = re.search(f"{username}/(.*)", locations[0]).group(1)
        location = f"~/my-private-bucket/{search}/gedi_subset.gpkg"

        return True, location
    else:
        return False

bool, location = get_result(result_job_id)

In [ ]:
# (Alternative) Call to maap.getJobResult without using the Backoff library
# Run cell multiple times until a response is received, the job may take some time to finish
import re

result = maap.getJobResult(result_job_id)
root = ET.fromstring(result.text)
locations = [element.text for element in root.findall(".//{http://www.opengis.net/wps/2.0}Data")]
print(locations)

search = re.search(f"{username}/(.*)", locations[0]).group(1)
location = f"~/my-private-bucket/{search}/gedi_subset.gpkg"

## 3. Verify the Results

Using the location set during the maap.getJobResult, plot the gpkg generated by the DPS job to view the results.

In [ ]:
# (Optional) Verify the results of the previously processed job using the `location` value set by the cells above
import geopandas as gpd
import matplotlib.pyplot as plt

gedi_gdf = gpd.read_file(location)
agbd_colors = plt.cm.get_cmap("viridis_r")
gedi_gdf.plot(column="agbd", cmap=agbd_colors);